# API do Olho Vivo

Danilo Lessa Bernardineli (http://fap.if.usp.br/~danlessa)

Esse notebook mostra rapidamente como acessar a API do Olhovivo.

A documentação do mesmo é encontrada em http://www.sptrans.com.br/desenvolvedores/APIOlhoVivo/Documentacao.aspx

O token pode ser obtido em http://www.sptrans.com.br/desenvolvedores/

## Dependências, definições e coisas globais

In [20]:
# Dependências

import pandas as pd
import numpy as np
import requests as req
import json
import pickle as pkl

In [2]:
# Constantes

token_acesso = "ab0855f97de16ac2b98e82920f6f07b1539ec2fc666d2b559303011bb057a25a"
url_api = "http://api.olhovivo.sptrans.com.br/v2.1"

In [3]:
# Coisas globais

cookies = None

## Exploração

### Autenticação

In [4]:
# Logar na SPTrans e ver o que rola

action_uri = url_api + "/Login/Autenticar?token={}".format(token_acesso)
r = req.post(action_uri, cookies=cookies)
cookies = r.cookies
print(r.text)

true


### Pesquisar linhas

In [21]:
# Logar na SPTrans e ver o que rola
action_uri = url_api + "/Linha/Buscar?termosBusca={}".format(8000)
r = req.get(action_uri, cookies=cookies)
output = r.text
print(r.text)

[{"cl":2506,"lc":false,"lt":"8000","sl":1,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":35274,"lc":false,"lt":"8000","sl":2,"tl":1,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":1273,"lc":false,"lt":"8000","sl":1,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"},{"cl":34041,"lc":false,"lt":"8000","sl":2,"tl":10,"tp":"PÇA. RAMOS DE AZEVEDO","ts":"TERM. LAPA"}]


In [28]:
output_json = json.loads(output)
print(output_json[0])

{'cl': 2506, 'lc': False, 'lt': '8000', 'sl': 1, 'tl': 1, 'tp': 'PÇA. RAMOS DE AZEVEDO', 'ts': 'TERM. LAPA'}


In [33]:
df = pd.DataFrame(output_json)
df

,cl,lc,lt,sl,tl,tp,ts
0,2506,False,8000,1,1,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
1,35274,False,8000,2,1,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
2,1273,False,8000,1,10,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
3,34041,False,8000,2,10,PÇA. RAMOS DE AZEVEDO,TERM. LAPA


In [35]:
df.columns = ["onibus", "a", "linha", "2", "3", "destino", "sentido"]

In [36]:
df

,onibus,a,linha,2,3,destino,sentido
0,2506,False,8000,1,1,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
1,35274,False,8000,2,1,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
2,1273,False,8000,1,10,PÇA. RAMOS DE AZEVEDO,TERM. LAPA
3,34041,False,8000,2,10,PÇA. RAMOS DE AZEVEDO,TERM. LAPA


### Pegar posições dos ônibus (linha)

In [57]:
linha = 466
action_uri = url_api + "/Posicao/Linha?codigoLinha={}".format(linha)
r = req.get(action_uri, cookies=cookies)
output = r.text
output_json = json.loads(output)
print(output)

{"hr":"14:35","vs":[{"p":"11096","a":false,"ta":"2018-06-20T17:32:08Z","py":-23.495889,"px":-46.6772735},{"p":"11100","a":false,"ta":"2018-06-20T17:33:22Z","py":-23.474276500000002,"px":-46.668260875},{"p":"11980","a":true,"ta":"2018-06-20T17:33:55Z","py":-23.5229225,"px":-46.664518},{"p":"11097","a":false,"ta":"2018-06-20T17:33:23Z","py":-23.535142,"px":-46.644700375},{"p":"11098","a":false,"ta":"2018-06-20T17:33:25Z","py":-23.528579999999998,"px":-46.661607}]}


In [58]:
output_json.keys()

dict_keys(['hr', 'vs'])

In [59]:
df = pd.DataFrame(output_json["vs"])

In [60]:
df.columns = ["Ida", "Onibus", "Longitude", "Latitude", "Timestamp"]
df["Linha"] = linha

In [61]:
df

,Ida,Onibus,Longitude,Latitude,Timestamp,Linha
0,False,11096,-46.677273,-23.495889,2018-06-20T17:32:08Z,466
1,False,11100,-46.668261,-23.474277,2018-06-20T17:33:22Z,466
2,True,11980,-46.664518,-23.522922,2018-06-20T17:33:55Z,466
3,False,11097,-46.644700,-23.535142,2018-06-20T17:33:23Z,466
4,False,11098,-46.661607,-23.528580,2018-06-20T17:33:25Z,466


### Pegar posições dos ônibus (geral)

In [62]:
# Logar na SPTrans e ver o que rola
action_uri = url_api + "/Posicao"
r = req.get(action_uri, cookies=cookies)
output = r.text
output_json = json.loads(output)
print(r.text[:100])

{"hr":"14:35","l":[{"c":"5011-10","cl":1115,"sl":1,"lt0":"STO. AMARO","lt1":"ELDORADO","qv":10,"vs":


In [63]:
output_json.keys()

dict_keys(['hr', 'l'])

In [64]:
pd.DataFrame(output_json["l"]).head(5)

,c,cl,lt0,lt1,qv,sl,vs
0,5011-10,1115,STO. AMARO,ELDORADO,10,1,"[{'p': 68430, 'a': True, 'ta': '2018-06-20T17:..."
1,978L-10,466,TERM. PRINC. ISABEL,TERM. CACHOEIRINHA,5,1,"[{'p': 11096, 'a': False, 'ta': '2018-06-20T17..."
2,5362-10,70,PÇA. DA SÉ,PQ. RES. COCAIA,13,1,"[{'p': 61615, 'a': True, 'ta': '2018-06-20T17:..."
3,675K-10,198,METRÔ STA. CRUZ,TERM. JD. ÂNGELA,12,1,"[{'p': 73877, 'a': True, 'ta': '2018-06-20T17:..."
4,6022-10,1207,STO. AMARO,PQ. CEREJEIRA,4,1,"[{'p': 78488, 'a': True, 'ta': '2018-06-20T17:..."


## Extração

In [76]:
output_posicoes = []

for linha in range(466, 476):
    action_uri = url_api + "/Posicao/Linha?codigoLinha={}".format(linha)
    r = req.get(action_uri, cookies=cookies)
    output = r.text
    output_json = json.loads(output)
    df = pd.DataFrame(output_json["vs"])
    if len(df) > 0 :
        df.columns = ["Ida", "Onibus", "Longitude", "Latitude", "Timestamp"]
        df["Linha"] = linha
        output_posicoes.append(df)

In [77]:
df = pd.concat(output_posicoes)

In [78]:
df

,Ida,Onibus,Longitude,Latitude,Timestamp,Linha
0,False,11096,-46.673250,-23.505985,2018-06-20T17:37:36Z,466
1,False,11100,-46.672950,-23.479852,2018-06-20T17:37:52Z,466
2,True,11980,-46.660823,-23.527275,2018-06-20T17:37:53Z,466
3,False,11098,-46.655797,-23.533821,2018-06-20T17:37:28Z,466
0,True,81891,-46.691208,-23.565766,2018-06-20T17:38:02Z,468
1,False,81826,-46.731731,-23.592173,2018-06-20T17:37:59Z,468
2,True,81866,-46.667170,-23.571322,2018-06-20T17:37:49Z,468
3,True,81031,-46.765423,-23.598582,2018-06-20T17:37:20Z,468
4,True,81219,-46.639628,-23.549518,2018-06-20T17:37:34Z,468
0,True,81981,-46.659599,-23.554470,2018-06-20T17:37:51Z,469
